In [ ]:
pip install pandas tqdm pillow faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.2 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-sp5vhhdp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-sp5vhhdp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import urllib.request
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

# Torch & CLIP
import torch
import clip

# Faiss
import faiss

# Metadata Embedding
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
import sqlite3
import pandas as pd
import json

conn = sqlite3.connect("/content/drive/MyDrive/data/db/cleaned_real_estate.db")
df = pd.read_sql("SELECT * FROM house", conn)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50840 entries, 0 to 50839
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            50840 non-null  int64  
 1   title               50840 non-null  object 
 2   price               50840 non-null  object 
 3   address             50840 non-null  object 
 4   floor               35268 non-null  float64
 5   deposit_type        50840 non-null  object 
 6   management_fee      41930 non-null  float64
 7   available_from      50840 non-null  object 
 8   house_num           50839 non-null  float64
 9   agent_comm          50673 non-null  float64
 10  agent_info          50840 non-null  object 
 11  rooms_count         50742 non-null  float64
 12  options             50840 non-null  object 
 13  posted_at           50840 non-null  object 
 14  gu                  50840 non-null  object 
 15  dong                50840 non-null  object 
 16  img_

###매물 데이터 불러오기

In [ ]:
# img_url 파싱 (JSON 문자열 → 리스트 → 대표(첫 번째) 이미지 추출)
def extract_first_img(json_str):
    try:
        urls = json.loads(json_str)
        return urls[0] if isinstance(urls, list) and urls else None
    except Exception:
        return None

df['img_path'] = df['img_url'].apply(extract_first_img)
df['has_image'] = df['img_path'].notnull()

###이미지 다운로드

In [ ]:
os.makedirs("images", exist_ok=True)

def download_image(url, path):
    try:
        urllib.request.urlretrieve(url, path)
        return True
    except:
        return False

# 이미지 다운로드
df['local_path'] = None
for idx, row in tqdm(df[df['has_image']].iterrows(), total=df['has_image'].sum()):
    save_path = f"images/{int(row['house_num'])}.jpg"
    if download_image(row['img_path'], save_path):
        df.at[idx, 'local_path'] = save_path
    else:
        df.at[idx, 'has_image'] = False

100%|██████████| 38997/38997 [2:36:43<00:00,  4.15it/s]


In [ ]:
## 🚨🚨 이미지 다운 다시 해야될 때 폴더 통째로 날리는 코드
# import shutil
# shutil.rmtree('/content/images')

clip의 역할 : 이미지 -> 임베딩(이미지의 특징을 고차원 숫자 벡터로 변환한 것)

이 임베딩을 벡터 db에 저장해서 faiss를 통해 유사한 값을 가진 이미지를 빠르게 검색함.

###이미지 임베딩 생성 (CLIP)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_embedding(image_path):
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            vec = model.encode_image(image)
            vec /= vec.norm(dim=-1, keepdim=True)
        return vec[0].cpu().numpy()
    except:
        return None

df['embedding'] = None
for idx, row in tqdm(df[df['has_image']].iterrows(), total=df['has_image'].sum()):
    vec = get_embedding(row['local_path'])
    if vec is not None:
        df.at[idx, 'embedding'] = vec
    else:
        df.at[idx, 'has_image'] = False

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 81.8MiB/s]
100%|██████████| 38997/38997 [08:59<00:00, 72.34it/s]


###메타데이터 벡터 생성

In [ ]:
numeric_features = ['deposit', 'monthly_rent', 'space']
categorical_features = ['gu', 'dong']

df[numeric_features] = df[numeric_features].fillna(0)
df[categorical_features] = df[categorical_features].fillna("없음")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

meta_features = preprocessor.fit_transform(df)

###최종 임베딩 결합 (img+메타데이터)

In [ ]:
final_embeddings = []

for idx, row in df.iterrows():
    meta_vec = meta_features[idx].toarray().flatten() if hasattr(meta_features[idx], 'toarray') else meta_features[idx]

    if row['has_image'] and row['embedding'] is not None:
        img_vec = row['embedding']
        combined_vec = np.concatenate([img_vec, meta_vec])
    else:
        # 이미지 없는 경우: 0벡터 패딩
        zero_img_vec = np.zeros(512)
        combined_vec = np.concatenate([zero_img_vec, meta_vec])

    final_embeddings.append(combined_vec)

In [ ]:
df['final_embedding'] = final_embeddings

###Faiss 인덱스 생성 및 저장

In [ ]:
embedding_dim = len(final_embeddings[0])
index = faiss.IndexFlatIP(embedding_dim)

embedding_matrix = np.vstack(final_embeddings).astype('float32')
faiss.normalize_L2(embedding_matrix)
index.add(embedding_matrix)

In [ ]:
# Faiss 인덱스 저장
faiss.write_index(index, "faiss_index.faiss")

###유사 매물 추천 함수

In [ ]:
def search_similar(query_house_num, top_k=5):
    row_match = df[df['house_num'] == query_house_num]
    if row_match.empty:
        print(f"[❗] house_num={query_house_num} 에 해당하는 매물이 없습니다.")
        return pd.DataFrame()

    row = row_match.iloc[0]

    query_vec = np.array(row['final_embedding'], dtype='float32').reshape(1, -1)
    faiss.normalize_L2(query_vec)

    D, I = index.search(query_vec, top_k)

    result = df.iloc[I[0]].copy()
    result['similarity'] = D[0]
    return result[['house_num', 'title', 'gu', 'dong', 'deposit', 'monthly_rent', 'space', 'similarity']]

In [ ]:
# 테스트
result = search_similar(2524646572, top_k=10)
print(result)

     house_num          title   gu dong    deposit  monthly_rent  space  \
0   2524646572  대치2단지 215동11층  강남구  개포동  250000000        800000     10   
21  2524372030  대치2단지 215동11층  강남구  개포동  250000000        800000     10   
117 2523326823  대치2단지 215동11층  강남구  개포동  250000000        800000     10   
76  2524290123  대치2단지 215동11층  강남구  개포동  250000000        800000     10   
75  2524245725  대치2단지 215동11층  강남구  개포동  250000000        800000     10   
93  2523444276           빌라3층  강남구  개포동  300000000        300000      9   
78  2524207790           기타3층  강남구  개포동  300000000        300000      8   
192 2524647306           빌라중층  강남구  개포동  300000000        300000      8   
90  2524041858       빌라 리치빌중층  강남구  개포동  300000000        300000      8   
74  2524297951           빌라3층  강남구  개포동  300000000        300000      8   

     similarity  
0             1  
21            1  
117           1  
76            1  
75            1  
93            1  
78            1  
192           1  
90          

In [ ]:
# 중복 제거
# title, address, gu, dong, price, area_size 모두 같으면 drop

In [ ]:
# 💡 다음 단계 (선택)
# 	•	중복 매물 제거: title, area_size, floor, price 조합으로 클러스터링
# 	•	API화 (FastAPI)
# 	•	임베딩 캐싱 (npy로 저장)
# 	•	이미지가 유사하면서도 지역, 가격, 평수도 유사한 하이브리드 필터 추가
# 	•	광각사진 탐지 모델 연동

In [ ]:
# 벡터 직렬화
df['final_embedding'] = df['final_embedding'].apply(
    lambda x: json.dumps(x.tolist()) if isinstance(x, np.ndarray) else json.dumps(x)
)

In [ ]:
import sqlite3

# 데이터베이스 연결
conn = sqlite3.connect('house_with_embedding.db')

# 데이터프레임을 SQLite 테이블로 저장
df.to_sql('house', conn, if_exists='replace', index=False)

# 연결 종료
conn.close()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50840 entries, 0 to 50839
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            50840 non-null  int64  
 1   title               50840 non-null  object 
 2   price               50840 non-null  object 
 3   address             50840 non-null  object 
 4   floor               35268 non-null  float64
 5   deposit_type        50840 non-null  object 
 6   management_fee      41930 non-null  float64
 7   available_from      50840 non-null  object 
 8   house_num           50839 non-null  float64
 9   agent_comm          50673 non-null  float64
 10  agent_info          50840 non-null  object 
 11  rooms_count         50742 non-null  float64
 12  options             50840 non-null  object 
 13  posted_at           50840 non-null  object 
 14  gu                  50840 non-null  object 
 15  dong                50840 non-null  object 
 16  img_